In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score,KFold

# 创建模型
model = GradientBoostingRegressor()

# 定义交叉验证的折数
n_folds = 10

# 创建KFold对象
kf = KFold(n_splits=n_folds, shuffle=True)

# 进行交叉验证并计算R²得分
scores = cross_val_score(model, X, y, cv=kf, scoring='r2')#neg_mean_squared_error

# 打印每个折的R²得分和平均得分
for fold, score in enumerate(scores):
    print(f"Fold {fold+1}: R² score = {score}")

print(f"Average R² score: {scores.mean()}")

In [ ]:
model = GradientBoostingRegressor()
model.fit(X, y)
# 获取特征重要性
importance = model.feature_importances_


# 将特征重要性与特征名对应起来
feature_importance = pd.DataFrame({'feature_name': X.columns, 'importance': importance})

# 按照重要性排序特征
feature_importance = feature_importance.sort_values('importance', ascending=False)

# 打印特征重要性排名
print(feature_importance[0:15])

In [ ]:
import shap
import matplotlib.pyplot as plt

# 训练模型
model = GradientBoostingRegressor()
model.fit(X, y)

# 计算 Shap 值
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 可视化 Shap 值
shap.summary_plot(shap_values, X, max_display=10,plot_size=(4,3.5))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


from sklearn.inspection import plot_partial_dependence

# 创建回归模型
model = GradientBoostingRegressor()
X_NameList=X.columns
# 拟合模型
model.fit(X, y)

for i in range(X.shape[1]):
    fig, ax = plt.subplots(figsize=(2, 1.5))
    plot_partial_dependence(model, X, features=[i], ax=ax, feature_names=X_NameList)
    ax.set_xlabel(X_NameList[i])


plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def read_COHP(fn):
    raw = open(fn).readlines()
    raw = [l for l in raw if 'No' not in l][3:]
    raw = [[eval(i) for i in l.split()] for l in raw]
    return np.array(raw)


data_cohp = read_COHP(r'COHPCAR.lobster')


for i in range(int((len(data_cohp[0])-3)/2)):
    fig, ax1 = plt.subplots(figsize=[3, 6])
    ax1.plot(-data_cohp[:, i*2+3], data_cohp[:, 0],
             color='k')#, label=labels_cohp[i])
    ax1.fill_betweenx(data_cohp[:, 0], -data_cohp[:, i*2+3], 0,
                      where=-data_cohp[:, i*2+3] >= 0, facecolor='green', alpha=0.4)
    ax1.fill_betweenx(data_cohp[:, 0], -data_cohp[:, i*2+3], 0,
                      where=-data_cohp[:, i*2+3] <= 0, facecolor='orange', alpha=0.4)
    ax1.set_ylim([-10, 10])
    ax1.set_yticks([-10, -5, 0, 5, 10])
    #ax1.set_xticks([ -3, 0, 3])
    ax1.yaxis.set_tick_params(labelright=True, right=True,labelleft=False, left=False,labelsize=18)
    ax1.xaxis.set_tick_params(labelsize=18)
    ax1.set_xlim([-2.9, 2.9])

    ax1.tick_params(axis='x', colors="k")
    # 添加图例，并指定色块
    ax1.legend(handles=[
        plt.Rectangle((0, 0), 1, 1, color='green', alpha=0.5, label='bonding' ),
        plt.Rectangle((0, 0), 1, 1, color='orange', alpha=0.5,label='anti-\nbonding'),
        
    ], loc='upper right', fontsize=10,ncol=1)

    # markers
    ax1.axvline(0, color='k', linestyle=':', alpha=0.5)
    ax1.axhline(0, color='k', linestyle='--', alpha=0.5)
    #fig.savefig(f'cohp-{i+1}.png', dpi=500,
               # bbox_inches="tight", transparent=True)
    #plt.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(6, 2))

data_800K = np.loadtxt(r"AIMD.dat")
time_800K = data_800K[:, 0]
tem_800K = data_800K[:, 1]
ener_800K = data_800K[:, 2]

# 对时间数据进行调整，使原点在纵坐标轴上
time_800K -= time_800K.min()

time_800K = time_800K[::4]
tem_800K = tem_800K[::4]
ener_800K = ener_800K[::4]
av = sum(ener_800K) / len(ener_800K)

# 绘制温度随时间的变化趋势
ax1.plot(time_800K, tem_800K, '#EE6363', label='Temperature', linewidth=0.8)
ax1.set_ylabel('Temperature (K)')
ax1.set_xlabel('Time (ps)')
ax1.tick_params(axis='y', labelcolor='#EE6363')
ax1.set_ylim(250, 2250)  # 调整左侧y轴范围
ax1.axhline(y=800, color='black', linestyle='--',alpha=0.6)

# 创建一个新的Axes对象并绘制能量随时间的变化趋势
ax2 = ax1.twinx()
ax2.plot(time_800K, ener_800K, '#4169E1', label='Energy', linewidth=0.8)
ax2.set_ylabel('Energy (eV)')
ax2.tick_params(axis='y', labelcolor='#4169E1')
ax2.set_ylim(-223, -215.5)  # 调整右侧y轴范围
ax2.axhline(y=av, color='black', linestyle='--',alpha=0.6)
ax1.set_xlim(0, 10000)


plt.title('Energy and Temperature Evolution')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 文件路径列表
file_paths = [
    r"PDOS_SUM_UP.dat",
    r"PDOS_SUM_DW.dat",
    r"PDOS_A_UP.dat",
    r"PDOS_A_DW.dat"
]
# 设定 Y 轴的范围
y_min = -5.5  # 最小值
y_max = 5.5 # 最大值

# 创建新的图形
fig, ax = plt.subplots(figsize=(3, 6))  # 设置图形的宽度和高度

colors = ['#4169E1','#4169E1','#EE6363','#EE6363','#2087C8', '#2087C8','#ffbd00', '#ffbd00']  # 自定义颜色列表

# 绘制数据
for i, file_path in enumerate(file_paths, start=1):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # 解析数据
    y_values = []
    x_values = []
    for line in lines:
        parts = line.split()
        y_values.append(float(parts[0]))  
        x_values.append(float(parts[1]))  
    
    # 绘制图形
    ax.plot(x_values, y_values, color=colors[i-1])

    # 添加渐变色
    ax.fill_betweenx(y_values, x_values, alpha=0.3, color=colors[i-1])

# 设置 y 轴的范围
ax.set_ylim(y_min, y_max)
ax.axhline(0, color='k', linestyle='--', alpha=0.5)
ax.xaxis.set_tick_params(labelsize=18)
ax.yaxis.set_tick_params(labelsize=18)
ax.set_xticks([ -3, 0, 3])
ax.set_xlim([-5, 5])


# 添加图例，并指定色块
ax.legend(handles=[
    plt.Rectangle((0, 0), 1, 1, color=colors[0], label='$NO_\mathrm{3}$-P'),
    plt.Rectangle((0, 0), 1, 1, color=colors[2], label='metal-d'),
], loc='upper right')

plt.show()